# All unit outcomes

Calculate the outcomes for every Small Area for each possible stroke unit.

## Aims

The results are eight tables, one for each stroke unit. Each table contains:
+ one row for each Small Area
+ columns for outcomes

## Method

load in fixed pathway times

add on to travel times from each small area to the unit

add on transfer times if necessary

so calculate total time to IVT and to MT for each Small Area

then feed these travel times into the stroke outcome model

and store some outcomes.

## Code setup

In [20]:
import pandas as pd
import os
from dataclasses import dataclass
import copy
import numpy as np

from stroke_outcome.continuous_outcome import Continuous_outcome
import stroke_outcome.outcome_utilities

In [47]:
# Define file paths
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''

    # Directories:
    dir_output = 'output'

    # Input data:
    df_treatment_times = 'sa_treatment_times.csv'
    
    # Output files:
    outcome_file_pre_str = 'sa_outcome_model_output_'

paths = Paths()

## Load treatment times

In [5]:
df_treatment_times = pd.read_csv(os.path.join(paths.dir_output, paths.df_treatment_times), index_col=0)

df_treatment_times.head()

,BT126BA_ivt,BT126BA_mt,BT161RH_ivt,BT161RH_mt,BT358DR_ivt,BT358DR_mt,BT412RL_ivt,BT412RL_mt,BT476SB_ivt,BT476SB_mt,BT521HS_ivt,BT521HS_mt,BT635QQ_ivt,BT635QQ_mt,BT746DN_ivt,BT746DN_mt
from_postcode,,,,,,,,,,,,,,,,
N00000001,131.400562,181.400562,137.852665,266.885323,164.223641,325.594875,119.032658,257.781276,179.895643,378.295643,161.926859,341.824797,136.580462,284.433127,202.7,416.7
N00000002,124.677730,174.677730,137.852665,266.885323,161.926859,323.298093,123.292278,262.040896,183.200000,381.600000,165.366046,345.263984,134.009674,281.862339,200.1,414.1
N00000003,124.677730,174.677730,137.852665,266.885323,163.077279,324.448513,120.471335,259.219953,180.991077,379.391077,163.077279,342.975217,135.299611,283.152276,201.4,415.4
N00000004,130.080264,180.080264,142.862547,271.895205,166.504588,327.875822,124.677730,263.426348,183.200000,381.600000,165.366046,345.263984,139.116622,286.969287,205.3,419.3
N00000005,132.710168,182.710168,140.372700,269.405358,166.504588,327.875822,116.088440,254.837059,176.591575,374.991575,158.450174,338.348112,140.372700,288.225365,206.6,420.6


Extract the list of stroke units:

In [6]:
unit_list = sorted(list(set([c.split('_')[0] for c in df_treatment_times.columns])))

unit_list

['BT126BA',
 'BT161RH',
 'BT358DR',
 'BT412RL',
 'BT476SB',
 'BT521HS',
 'BT635QQ',
 'BT746DN']

## Calculate outcomes

Do this separately for each stroke unit.

Calculate outcomes for these cohorts of patients:
+ nLVO with IVT
+ LVO with IVT
+ LVO with MT

Then combine the results into these groups:
+ RACE < 5, treated population only
+ RACE >= 5, treated population only
+ Full population, actual treatment rates
+ Full population, target treatment rates



### Set up base inputs for outcome model

In [7]:
patient_cohorts = {
    'nlvo_ivt': {
        'stroke_type_code': 1,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 0,
    },
    'lvo_ivt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 0,
    },
    'lvo_ivt_mt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 1,
        'mt_chosen_bool': 1,
    },
    'lvo_mt': {
        'stroke_type_code': 2,
        'ivt_chosen_bool': 0,
        'mt_chosen_bool': 1,
    },
}

Load reference mRS distributions to find the proportions of patients with mRS<=2 given no treatment:

In [21]:
mrs_dists = stroke_outcome.outcome_utilities.import_mrs_dists_from_file()

mrs_dists

,mRS<=0,mRS<=1,mRS<=2,mRS<=3,mRS<=4,mRS<=5,mRS<=6
Stroke type,,,,,,,
pre_stroke_nlvo,0.583,0.746,0.850,0.951,0.993,1.000,1
pre_stroke_lvo,0.408,0.552,0.672,0.838,0.956,1.000,1
no_treatment_lvo,0.050,0.129,0.265,0.429,0.676,0.811,1
no_treatment_nlvo,0.198,0.460,0.580,0.708,0.856,0.918,1
no_effect_nlvo_ivt_deaths,0.196,0.455,0.574,0.701,0.847,0.908,1
no_effect_lvo_ivt_deaths,0.048,0.124,0.255,0.414,0.653,0.783,1
no_effect_lvo_mt_deaths,0.048,0.124,0.255,0.412,0.649,0.779,1
t0_treatment_nlvo_ivt,0.445,0.642,0.752,0.862,0.941,0.967,1
t0_treatment_lvo_ivt,0.140,0.233,0.361,0.522,0.730,0.838,1


In [22]:
no_treatment_nlvo_mrsleq2 = mrs_dists.loc['no_treatment_nlvo', 'mRS<=2']
no_treatment_lvo_mrsleq2 = mrs_dists.loc['no_treatment_lvo', 'mRS<=2']

### Run outcome model

In [11]:
# Set up outcome model
outcome_model = Continuous_outcome()

In [31]:
def run_outcomes(
        ivt_times,
        mt_times,
        patient_cohorts,
        no_treatment_nlvo_mrsleq2,
        no_treatment_lvo_mrsleq2,
        index_names=[]
    ):
    # Input data for outcome model:
    df_patients = pd.DataFrame()
    df_patients['onset_to_needle_mins'] = ivt_times
    df_patients['onset_to_puncture_mins'] = mt_times
    
    # Store averaged outcomes in here:
    df_results = pd.DataFrame()
    if len(index_names) > 0:
        df_results.index = index_names
    else:
        pass

    for cohort_name, cohort_dict in patient_cohorts.items():
        # Assign patient details:
        df_patients['stroke_type_code'] = cohort_dict['stroke_type_code']
        df_patients['ivt_chosen_bool'] = cohort_dict['ivt_chosen_bool']
        df_patients['mt_chosen_bool'] = cohort_dict['mt_chosen_bool']
    
        # Run outcomes:
        outcome_model.assign_patients_to_trial(df_patients)
        
        # Calculate outcomes:
        patient_data_dict, outcomes_by_stroke_type, full_cohort_outcomes = (
            outcome_model.calculate_outcomes())
        
        # Make a copy of the results:
        outcomes_by_stroke_type = copy.copy(outcomes_by_stroke_type)
        full_cohort_outcomes = copy.copy(full_cohort_outcomes)
        
        # Place the relevant results into the results dataframe.
        # Round them to 5 decimal places which should be plenty.
        df_results[f'{cohort_name}_added_utility'] = np.round(
            full_cohort_outcomes['each_patient_utility_shift'], 5)
        df_results[f'{cohort_name}_mean_mrs'] = np.round(
            full_cohort_outcomes['each_patient_mrs_post_stroke'], 5)
        df_results[f'{cohort_name}_mrs_less_equal_2'] = np.round(
            full_cohort_outcomes['each_patient_mrs_dist_post_stroke'][:, 2], 5)
        df_results[f'{cohort_name}_mrs_shift'] = np.round(
            full_cohort_outcomes['each_patient_mrs_shift'], 5)        

        # Calculate the shift in the proportion mRS<=2
        # compared with the no-treatment population.
        if cohort_dict['stroke_type_code'] == 1:
            no_treatment_mrsleq2 = no_treatment_nlvo_mrsleq2
        else:
            no_treatment_mrsleq2 = no_treatment_lvo_mrsleq2

        # Round the results again to avoid floating point errors.
        df_results[f'{cohort_name}_added_mrs_less_equal_2'] = (
            np.round((
                df_results[f'{cohort_name}_mrs_less_equal_2'] -
                no_treatment_mrsleq2
            ), 5))

    return df_results

In [32]:
dict_results = {}

for first_stroke_unit in unit_list:
    df_results = run_outcomes(
        df_treatment_times[f'{first_stroke_unit}_ivt'],
        df_treatment_times[f'{first_stroke_unit}_mt'],
        patient_cohorts,
        no_treatment_nlvo_mrsleq2,
        no_treatment_lvo_mrsleq2,
        index_names=df_treatment_times.index
    )
    dict_results[first_stroke_unit] = df_results

View the results for the first stroke unit in the list:

In [33]:
dict_results[unit_list[0]].head(3).T

from_postcode,N00000001,N00000002,N00000003
nlvo_ivt_added_utility,0.10711,0.10988,0.10988
nlvo_ivt_mean_mrs,1.69309,1.67704,1.67704
nlvo_ivt_mrs_less_equal_2,0.69579,0.69884,0.69884
nlvo_ivt_mrs_shift,-0.58691,-0.60296,-0.60296
nlvo_ivt_added_mrs_less_equal_2,0.11579,0.11884,0.11884
lvo_ivt_added_utility,0.05214,0.05399,0.05399
lvo_ivt_mean_mrs,3.38255,3.37244,3.37244
lvo_ivt_mrs_less_equal_2,0.32186,0.32381,0.32381
lvo_ivt_mrs_shift,-0.25745,-0.26756,-0.26756
lvo_ivt_added_mrs_less_equal_2,0.05686,0.05881,0.05881


## Save results to file

In [50]:
for unit_name, df in dict_results.items():
    path_to_file = os.path.join(paths.dir_output, paths.outcome_file_pre_str + unit_name + '.csv')
    df.to_csv(path_to_file)